```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. 

```

# Math - Linear Algebra

In [1]:
# Connect with underlying Python code
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../src')

In [2]:
from datasets import (
    get_dataset,
)

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

- Linear combinations
- Weighted Average
- Sums of Squares
- Sum of Squared Errors

## Linear Combinations

When mathematical folks talk about a `linear combination`, they are using a technical term for what we do when we check out from the grocery store. If your grocery store bill looks like:
```
Product      Quantity     Cost Per
----------------------------------
Wine         2            12.50
Orange       12           .50
Muffin       3            1.75
```
you can figure out the `total cost` with some arithmetic:

In [4]:
(2 * 12.50) + (12 * .5) + (3 * 1.75)

36.25

We might think of this as a `weighted sum`. A `sum` by itself is simply adding things up. The total number of items we bought is:

In [5]:
2 + 12 + 3

17

However, when we buy things, we pay for each item based on its cost. To get a total
cost, we have to add up a sequence of costs times quantities. I can phrase that in a slightly different way: `we have to weight the quantities of different items by their respective prices`.

In [6]:
# pure python, old-school
quantity = [2, 12, 3]
costs = [12.5, .5, 1.75]
partial_cost = []

for q, c in zip(quantity, costs):
    partial_cost.append(q*c)

sum(partial_cost)

36.25

In [7]:
# pure python, for the new-school, cool kids
quantity = [2, 12, 3]
costs = [12.5, .5, 1.75]

sum(q * c for q, c in zip(quantity, costs))

36.25

Let’s return to computing the total cost. If I line up the quantities and costs in NumPy arrays, I can run the same calculation. I can also get the benefits of data that is more organized under the hood, concise code that is easily extendible for more quantities and costs, and better small- and large-scale performance:

In [8]:
quantity = np.array([2, 12, 3])
costs = np.array([12.5, .5, 1.75])

np.sum(quantity * costs) # element-wise multiplication

36.25

This calculation can also be performed by NumPy with `np.dot`. `dot` multiplies the
elements pairwise, selecting the pairs in lockstep down the two arrays, and then adds them up:

In [9]:
print(quantity.dot(costs),     # dot-product way 1
      np.dot(quantity, costs), # dot-product way 2
      quantity @ costs,        # dot-product way 3
                               # (new addition to the family!)
      sep='\n')

36.25
36.25
36.25


There are two things that make the `linear combination` (expressed in a `dot
product`): (1) we multiply the values pairwise, and (2) we add up all those subresults. These correspond to (1) a single multiplication to create subtotals for each line on a receipt and (2) adding those subtotals together to get your final bill.

You’ll also see the dot product written mathematically (using q for quantity and c for cost) as $\sum \limits _{i} q_{i}c_{i}$. If you haven’t seen this notation before, here’s a breakdown:
1. The $\sum$, a capital Greek sigma, means add up,
2. The $q_{i}c_{i}$ means multiply two things, and
3. The i ties the pieces together in lockstep like a sequence index.

More briefly, it says, “add up all of the element-wise multiplied q and c.” Even more briefly, we might call this `the sum product of the quantities and costs`. At our level, we can use sum product as a synonym for `dot product`.
So, combining NumPy on the left-hand side and mathematics on the right-hand side,
we have:

np.dot(quantity, cost) = $\sum \limits _{i} q_{i}c_{i}$

Sometimes, that will be written as briefly as qc. If I want to emphasize the dot product, or remind you of it, I’ll use a bullet (•) as its symbol: q • c. If you are uncertain about the element-wise or lockstep part, you can use Python’s zip function to help you out. It is designed precisely to march, in lockstep, through multiple sequences.

In [11]:
for q_i, c_i in zip(quantity, costs):
    print("{:2d} {:6.2f} --> {:5.2f}".format(q_i, c_i, q_i * c_i))

print("Total:", sum(q*c for q,c in zip(quantity, costs))) # cool-kid method

 2  12.50 --> 25.00
12   0.50 -->  6.00
 3   1.75 -->  5.25
Total: 36.25


## Weighted Average
The `simple average` — also called the `mean` — is an `equally weighted average` computed from a set of values. For example, if I have three values (10, 20, 30), I divide up my weights equally among the three values and, presto, I get thirds: 
```
1/3*10 + 1/3*20 + 1/3*30
```
You might be looking at me with a distinct side eye, but if I rearrange that as (10+20+30)/3 you might be happier. I simply do sum(values)/3: add them all
up and divide by the number of values. Look what happens, however, if I go back to the more expanded method:

In [14]:
values = np.array([10.0, 20.0, 30.0])
weights = np.full_like(values, 1/3) # repeated (1/3)

print("weights:", weights)
print("via mean:", np.mean(values))
print("via weights and dot:", np.dot(weights, values))
print("via weights and dot:", weights.dot(values))
print("via weights and dot:", weights @ values)

weights: [0.33333333 0.33333333 0.33333333]
via mean: 20.0
via weights and dot: 20.0
via weights and dot: 20.0
via weights and dot: 20.0


We can write the mean as a weighted sum — a sum product between values and weights. If we start playing around with the weights, we end up with the concept of `weighted averages`. With weighted averages, instead of using equal portions, we break the portions up any way we choose. In some scenarios, we insist that the portions add up to one. Let’s say we weighted our three values by 1/2, 1/4, 1/4. Why might we do this? These weights could express the idea that the first option is valued twice as much as the other two and that the other two are valued equally. It might also mean that the first one is twice as likely in a random scenario. These two interpretations are close to what we would get if we applied those weights to underlying costs or quantities. You can view them as two sides of the same double-sided coin.

In [17]:
values = np.array([10, 20, 30])
weights = np.array([.5, .25, .25])

np.dot(weights, values)

17.5

One special weighted average occurs when the values are the different outcomes of a
random scenario and the weights represent the probabilities of those outcomes. In this case, the weighted average is called the `expected value` of that set of outcomes. 

Here’s a simple game. Suppose I roll a standard six-sided die and I get USD 1.00 if the die turns out odd and I lose USD .50 if the die comes up even. Let’s compute a dot product between the payoffs and the probabilities of each payoff. My expected outcome is to make:

In [18]:
                  # odd, even
payoffs = np.array([1.0, -.5])
probs = np.array([ .5, .5])

np.dot(payoffs, probs)

0.25

Mathematically, we write the expected value of the game as 

E(game) = $\sum \limits _{i} p_{i}v_{i}$

with `p` being the probabilities of the events and `v` being the values or payoffs of those events. 

Now, in any single run of that game, I’ll either make USD 1.00 or lose USD .50. But, if I were to play the game, say 100 times, I’d expect to come out ahead by about USD 25.00 — the expected gain per game times the number of games. In reality, this outcome is a random event. Sometimes, I’ll do better. Sometimes, I’ll do worse. But USD 25.00 is my best guess before heading into a game with 100 tosses. With many, many tosses, we’re highly likely to get very close to that `expected value`.

Here’s a simulation of 10000 rounds of the game. You can compare the outcome:

In [19]:
# The expected value
np.dot(payoffs, probs) * 10000

2500.0

In [22]:
# Game simulation

def is_even(n):
    # if remainder 0, value is even
    return n % 2 == 0

winnings = 0.0

for toss_ct in range(10000):
    die_toss = np.random.randint(1, 7)
    winnings += 1.0 if is_even(die_toss) else -0.5

print(winnings)

2552.5


## Sums of Squares
One other, very special, sum-of-products is when both the quantity and the value are two copies of the same thing. For example, 
```
5 · 5 + (−3)·(−3) + 2 · 2 + 1 · 1 = 
52 + 32 + 22 + 12 = 
25 + 9 + 4 + 1 = 
39
```
This is called a `sum of squares` since each element, multiplied by itself, gives the square of the original value. Here is how we can do that in code:

In [23]:
values = np.array([5, -3, 2, 1])
squares = values * values # element-wise multiplication

print(squares,
      np.sum(squares),        # sum of squares
      np.dot(values, values), 
      sep="\n")

[25  9  4  1]
39
39


If I wrote this mathematically, it would look like: 

dot(values, values) = $\sum \limits _{i} v_{i}v_{i}$ = $\sum \limits _{i} v^{2}_{i}$

## Sum of Squared Errors
